In [1]:
import pandas as pd
from utils import grid_df_display
from tqdm.notebook import tqdm
import numpy as np
import copy

In [2]:
df = pd.read_csv('./Data/contacts.csv', 
                 dtype = {'Id': int,'Email': str,'Phone': str, 'Contacts': int, 'OrderId': str},
                 engine = 'c')

In [3]:
df

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,NaN,1,NaN
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,NaN,9125983679,0,NaN
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,NaN,300364407,2,NaN
...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku


<img src="./Fig/explanation.png" alt="Drawing" style="width: 1000px;"/>

### Algorithm
#### Step 1
* Find which `ID` has the same `Email.
* Find which `ID` has the same `Phone`. 
* Find which `ID` has the same `OrderId`.

#### Step 2
* Find connection `ID` by using step 1 result in each row.

#### Step 3
*  Create a dictionary where keys are `ID` and values are set of connection `ID`.

#### Step 4
* Create an empty dictionary called `dic`.
* Iterate over items in Step 3's dictionary. ==> It will return a set of connection `ID` step by step.
    * Iterate over connection `ID` in the set.
        * If `ID` is already in the dictionary, use a list to record `ID`.
    * If the list's length is bigger than 0.
       * Using `ID`(key), recorded before, to find each value(set of connection `ID`).
       * And merge each set we found before and the set of connection `ID` we Iterate this round.
    * Else, update items by the set of connection `ID` in this round. Each value in the set will be set to the key and copy the set to become each key value.
      And then update to the `dic` we create at first.

In [4]:
Email_id = df.groupby('Email').agg({'Id': set}).rename(columns={'Id': 'same_email_id'}).reset_index()
Phone_id = df.groupby('Phone').agg({'Id': set}).rename(columns={'Id': 'same_phone_id'}).reset_index()
OrderId_id = df.groupby('OrderId').agg({'Id': set}).rename(columns={'Id': 'same_orderid_id'}).reset_index()

In [5]:
df = df.merge(Email_id, how='left', on='Email')
df = df.merge(Phone_id, how='left', on='Phone')
df = df.merge(OrderId_id, how='left', on='OrderId')

In [6]:
df[['same_email_id', 'same_phone_id', 'same_orderid_id']] = df[['same_email_id', 'same_phone_id', 'same_orderid_id']].replace({np.nan: set()})

In [7]:
def merge_column_value(x):
    s = set()
    s.update(x['same_email_id'])
    s.update(x['same_phone_id'])
    s.update(x['same_orderid_id'])
    return s

In [8]:
df['group_id'] = df[['same_email_id', 'same_phone_id', 'same_orderid_id']].apply(merge_column_value, axis=1)
df['group_id_count'] = df['group_id'].apply(len)

In [9]:
df.head()

,Id,Email,Phone,Contacts,OrderId,same_email_id,same_phone_id,same_orderid_id,group_id,group_id_count
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},{},{},{0},1
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,{},{1},"{476346, 1, 2458}","{1, 476346, 2458}",3
2,2,NaN,9125983679,0,NaN,{},"{2, 348955}",{},"{2, 348955}",2
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},{},{3},{3},1
4,4,NaN,300364407,2,NaN,{},{4},{},{4},1


In [10]:
df = df.drop(['same_email_id', 'same_phone_id', 'same_orderid_id'], axis=1)

In [11]:
df.head()

,Id,Email,Phone,Contacts,OrderId,group_id,group_id_count
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},1
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 476346, 2458}",3
2,2,NaN,9125983679,0,NaN,"{2, 348955}",2
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},1
4,4,NaN,300364407,2,NaN,{4},1


In [12]:
original_dic = copy.deepcopy(dict(zip(df['Id'], df['group_id'])))

In [13]:
dic = {}
for _, group_id in tqdm(original_dic.items()):
    li=[]
    for _id in group_id:
        if _id in dic:
            li.append(_id)
        else:
            pass
    if len(li)>0:
        s = set()
        [s.update(dic[i]) for i in li]
        s.update(group_id)
    else:
        s = group_id.copy()
    dic.update(
            dict( zip( s, [s]*len(s) ) )
        )         

  0%|          | 0/500000 [00:00<?, ?it/s]

In [14]:
data = [[dic[i]] for i in range(len(dic))]
result = pd.DataFrame(data)

In [15]:
result.columns = ['result']

In [16]:
df = pd.concat([df, result], axis=1)

In [17]:
df

,Id,Email,Phone,Contacts,OrderId,group_id,group_id_count,result
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},1,{0}
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 476346, 2458}",3,"{1, 165605, 2458, 140081, 115061, 98519, 476346}"
2,2,NaN,9125983679,0,NaN,"{2, 348955}",2,"{2, 322639, 159312, 348955}"
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},1,{3}
4,4,NaN,300364407,2,NaN,{4},1,{4}
...,...,...,...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,{499995},1,{499995}
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,{499996},1,{499996}
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,{499997},1,{499997}
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,"{499998, 121111}",2,"{499998, 121111}"


In [18]:
del original_dic
del dic
del data

In [19]:
df['result'] = df['result'].apply(list)

In [20]:
def sort_list_value_and_convert_value_to_string(x):
    x = sorted(x)
    return '-'.join([str(i) for i in x])

In [21]:
df['result'] = df['result'].apply(sort_list_value_and_convert_value_to_string)

In [22]:
by_result = df.groupby('result').agg({'Contacts': 'sum'}).reset_index()
by_result['Contacts'] = by_result['Contacts'].apply(str)

In [23]:
by_result['final_result'] = by_result['result'] + ', ' + by_result['Contacts']

In [24]:
by_result

,result,Contacts,final_result
0,0,1,"0, 1"
1,1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
2,10-93270,7,"10-93270, 7"
3,100-822-1157-79530-95287-109959-129043-146402-...,38,100-822-1157-79530-95287-109959-129043-146402-...
4,1000-86254,3,"1000-86254, 3"
...,...,...,...
291899,99994,0,"99994, 0"
291900,99995,3,"99995, 3"
291901,99996,1,"99996, 1"
291902,99998-125369,4,"99998-125369, 4"


In [25]:
df = df.merge(by_result, on='result', how='left')

In [26]:
df

,Id,Email,Phone,Contacts_x,OrderId,group_id,group_id_count,result,Contacts_y,final_result
0,0,gkzAbIy@qq.com,NaN,1,NaN,{0},1,0,1,"0, 1"
1,1,NaN,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,"{1, 476346, 2458}",3,1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,NaN,9125983679,0,NaN,"{2, 348955}",2,2-159312-322639-348955,4,"2-159312-322639-348955, 4"
3,3,mdllpYmE@gmail.com,NaN,0,bHquEnCbbsGLqllwryxPsNOxa,{3},1,3,0,"3, 0"
4,4,NaN,300364407,2,NaN,{4},1,4,2,"4, 2"
...,...,...,...,...,...,...,...,...,...,...
499995,499995,NaN,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,{499995},1,499995,2,"499995, 2"
499996,499996,NaN,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,{499996},1,499996,4,"499996, 4"
499997,499997,NaN,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,{499997},1,499997,2,"499997, 2"
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,"{499998, 121111}",2,121111-499998,5,"121111-499998, 5"


### Check Final Result

In [27]:
filter_li = [14,65749,83313,267520,323910,359095]
df.query("Id == @filter_li")

,Id,Email,Phone,Contacts_x,OrderId,group_id,group_id_count,result,Contacts_y,final_result
14,14,NaN,17423803380,4,NaN,"{323910, 14}",2,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"
65749,65749,EpFjqcrumip@yahoo.com,NaN,1,uwUOTlevAkzhnIqGgWHFsQKzO,"{267520, 65749}",2,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"
83313,83313,NaN,2802165027,0,NaN,"{267520, 83313}",2,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"
267520,267520,sXHosuHvxkP@gmail.com,2802165027,2,uwUOTlevAkzhnIqGgWHFsQKzO,"{267520, 83313, 65749, 359095}",4,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"
323910,323910,TFEFqCcFHnW@yahoo.com,17423803380,2,rbrdWpswvYIwIWIWOppvtluAK,"{359095, 323910, 14}",3,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"
359095,359095,sXHosuHvxkP@gmail.com,NaN,1,rbrdWpswvYIwIWIWOppvtluAK,"{267520, 323910, 359095}",3,14-65749-83313-267520-323910-359095,10,"14-65749-83313-267520-323910-359095, 10"


In [28]:
filter_li = [33,459,46643,117352,269224,499013]
df.query("Id == @filter_li")

,Id,Email,Phone,Contacts_x,OrderId,group_id,group_id_count,result,Contacts_y,final_result
33,33,XLAVlZUqxWprVG@gmail.com,NaN,3,VFfXBTpVBgmCPBNSXsnckcwMM,"{33, 459, 499013}",3,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
459,459,hCzxVKHvKB@yahoo.com,721136055,2,VFfXBTpVBgmCPBNSXsnckcwMM,"{33, 499013, 117352, 459}",4,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
46643,46643,xaeXpbCVX@hotmail.com,NaN,3,mFRWUrCVkmgYQpzCklCprmLSU,"{269224, 46643}",2,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
117352,117352,hCzxVKHvKB@yahoo.com,NaN,1,IWFKFTijQDxXGyWSrMwzbIOBI,"{117352, 459}",2,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
269224,269224,NaN,40846711002,0,mFRWUrCVkmgYQpzCklCprmLSU,"{269224, 46643, 499013}",3,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
499013,499013,AapUeVZEmikwXrThUncp@hotmail.com,40846711002,4,VFfXBTpVBgmCPBNSXsnckcwMM,"{33, 499013, 269224, 459}",4,33-459-46643-117352-269224-499013,13,"33-459-46643-117352-269224-499013, 13"
